# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- **population** - The population as of 2015.
- **population_growth** - The annual population growth rate, as a percentage.
- **area** - The total land and water area.

First Let's connect to the database 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

I am going to list the tables of the database, and look quickly to the main table content.

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT * 
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- **name** - The name of the country.
- **area** - The country's total area (both land and water).
- **area_land** - The country's land area in square kilometers.
- **area_water** - The country's waterarea in square kilometers.
- **population** - The country's population.
- **population_growth** - The country's population growth as a percentage.
- **birth_rate** - The country's birth rate, or the number of births a year per 1,000 people.
- **death_rate** - The country's death rate, or the number of death a year per 1,000 people.

## Calculate some summary statistics

Let's start by calculating some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT 
    MIN(population) AS Min_population,
    MAX(population) AS Max_population,
    MIN(population_growth) AS Min_population_growth,
    MAX(population_growth) AS Max_population_growth
FROM facts;

 * sqlite:///factbook.db
Done.


Min_population,Max_population,Min_population_growth,Max_population_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)
Let's use subqueries to zoom in on just these countries.

In [5]:
%%sql
SELECT 
    name min_population_country, 
    population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


min_population_country,population
Antarctica,0


In [6]:
%%sql
SELECT name max_population_country
FROM facts
WHERE population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


max_population_country
World


The result of queries shows facts table contains the whole world information and also the information about Antarctica which is uninhabited.

So we should recalculate the summary statistics we calculated earlier.

In [7]:
%%sql
SELECT 
   MIN(population) AS Min_population,
   MAX(population) AS Max_population,
   MIN(population_growth) AS Min_population_growth,
   MAX(population_growth) AS Max_population_growth
FROM facts
WHERE name NOT IN ('World', 'Antarctica');

 * sqlite:///factbook.db
Done.


Min_population,Max_population,Min_population_growth,Max_population_growth
48,1367485388,0.0,4.02


It is also good to know about the average value for the following columns:
- population
- area

In [8]:
%%sql
SELECT 
    AVG(population) AS Avg_population,
    AVG(area) AS Avg_area
FROM facts
WHERE name NOT IN ('World', 'Antarctica');

 * sqlite:///factbook.db
Done.


Avg_population,Avg_area
32377011.0125,555093.546184739


## Densely populated countries

It is intersting to identify densely populated countries that have:

- Above average values for population.
- Below average values for area.

In [9]:
%%sql
SELECT name densely_populated_country
FROM facts
WHERE population > (
                    SELECT AVG(population) AS Avg_population              
                    FROM facts
                   )
        AND area < (
                    SELECT AVG(area) AS Avg_area
                    FROM facts
                   );

 * sqlite:///factbook.db
Done.


densely_populated_country
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


## Countries with the most population and population growth:

let's see what country has **the most people** and what country has the **highest growth rate**?

In [10]:
%%sql
SELECT 
    name Max_population_country, 
    population, population_growth
FROM facts
WHERE population = (
                    SELECT MAX(population) AS Max_population
                    FROM facts
                    WHERE name not in ('World', 'Antarctica')
                   );

 * sqlite:///factbook.db
Done.


Max_population_country,population,population_growth
China,1367485388,0.45


In [11]:
%%sql
SELECT 
    name Max_population_growth_country, 
    population, population_growth
FROM facts
WHERE population_growth = (
                           SELECT MAX(population_growth) AS Max_population_growth
                           FROM facts
                           WHERE name not in ('World', 'Antarctica')
                          );

 * sqlite:///factbook.db
Done.


Max_population_growth_country,population,population_growth
South Sudan,12042910,4.02


**China** with more than 1.3 billion people has the most population but the population growth is only 0.45 while **South Sudan** with a population of 1.2 million has the highest population growth which is 4.02.

## Highest ratios of water to land:
Let's find the answers to the following questions:

- Which countries have the highest ratios of water to land? 
- Which countries have more water than land?

In [12]:
%%sql
SELECT 
    name, 
    CAST(area_water AS Float)/CAST(area_land AS float) AS water_land_ratio
FROM facts
WHERE CAST(area_water AS Float)/CAST(area_land AS float) = 
           (
            SELECT MAX(CAST(area_water AS Float)/CAST(area_land AS float))
            FROM facts
            WHERE name not in ('World', 'Antarctica')
           );

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666


The result is **British Indian Ocean Territory** which is a part of the Indian ocean so it is better to see the 10 top countries with respect to the water and land ration.

In [13]:
%%sql
SELECT name, area_water, area_land,
       CAST(area_water AS Float)/CAST(area_land AS float) AS water_land_ratio
FROM facts
WHERE name not in ('World', 'Antarctica')
ORDER BY water_land_ratio desc
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,area_water,area_land,water_land_ratio
British Indian Ocean Territory,54340,60,905.6666666666666
Virgin Islands,1564,346,4.520231213872832
Puerto Rico,4921,8870,0.5547914317925592
"Bahamas, The",3870,10010,0.3866133866133866
Guinea-Bissau,8005,28120,0.2846728307254623
Malawi,24404,94080,0.25939625850340137
Netherlands,7650,33893,0.22571032366565366
Uganda,43938,197100,0.22292237442922375
Eritrea,16600,101000,0.16435643564356436
Liberia,15049,96320,0.15623961794019933


After some of the Caribean islands like **Virgin Islands**, **Puerto Rico**, and **The Bahamas**, **Guinea-Bissau** is the country with the highest ratio of water to land. Next are **Malawi** and **Netherlands** respectively.

According to the above table, **British Indian Ocean Territory** and **Virgin Islands** have more areas of water than land.

## Most population next year:
In this section, I am going to figure out which countries will add the most people to their population next year. As we saw before South Saudan has the post population growth but its population is only 1.2 million so I am going to estimate the next year added population for each country by multiplying *the percentage of population growth* and *population*. Then I can find out which countries add the most people. 

In [14]:
%%sql
SELECT 
    name, 
    population, 
    population_growth,
    CAST(population_growth / 100 * population AS Int) added_population
FROM facts
WHERE name not in ('World', 'Antarctica') 
ORDER BY added_population DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,population_growth,added_population
India,1251695584,1.22,15270686
China,1367485388,0.45,6153684
Nigeria,181562056,2.45,4448270
Pakistan,199085847,1.46,2906653
Ethiopia,99465819,2.89,2874562
Bangladesh,168957745,1.6,2703323
United States,321368864,0.78,2506677
Indonesia,255993674,0.92,2355141
"Congo, Democratic Republic of the",79375136,2.45,1944690
Philippines,100998376,1.61,1626073


**India** is the country which will add the most people to the world which is going to be about 15 millions. Although **China**'s current population is more than India but because of the significant low population growth, the number of added people will be about 6 millions which is significantly less than India but it is still in the second position. **Nigeria** is in the third position and after that there are **Pakistan** and **Ethiopia**.

## Higher death rate than birth rate:
It is interesting to find out which countries have a higher death rate than birth rate.

In [15]:
%%sql
SELECT 
    name, 
    birth_rate, death_rate, population_growth, 
    ROUND(death_rate - birth_rate, 2) AS net_death_rate
FROM facts
WHERE name NOT IN ('World', 'Antarctica') AND
      death_rate > birth_rate
ORDER BY net_death_rate DESC;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate,population_growth,net_death_rate
Bulgaria,8.92,14.44,0.58,5.52
Serbia,9.08,13.66,0.46,4.58
Latvia,10.0,14.31,1.06,4.31
Lithuania,10.1,14.27,1.04,4.17
Ukraine,10.72,14.46,0.6,3.74
Hungary,9.16,12.73,0.22,3.57
Germany,8.47,11.42,0.17,2.95
Slovenia,8.42,11.37,0.26,2.95
Romania,9.14,11.9,0.3,2.76
Croatia,9.45,12.18,0.13,2.73


The above countries have a larger death rate than birth rate so it seems they should have negative population growth. I am going to figure out if the migration rate keeps the population growth rate positive. 
I am going to add a new column next to the population growth which is the calculated population growth, it makes the comparison easier.

In [16]:
%%sql
SELECT 
    name, 
    birth_rate, 
    death_rate, 
    migration_rate, 
    population_growth, 
    ROUND((birth_rate + migration_rate - death_rate) /10, 2) cal_pop_growth,
    ROUND(death_rate - birth_rate, 2) AS net_death_rate
FROM facts
WHERE name NOT IN ('World', 'Antarctica') AND
      death_rate > birth_rate
ORDER BY net_death_rate DESC;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate,migration_rate,population_growth,cal_pop_growth,net_death_rate
Bulgaria,8.92,14.44,0.29,0.58,-0.52,5.52
Serbia,9.08,13.66,0.0,0.46,-0.46,4.58
Latvia,10.0,14.31,6.26,1.06,0.19,4.31
Lithuania,10.1,14.27,6.27,1.04,0.21,4.17
Ukraine,10.72,14.46,2.25,0.6,-0.15,3.74
Hungary,9.16,12.73,1.33,0.22,-0.22,3.57
Germany,8.47,11.42,1.24,0.17,-0.17,2.95
Slovenia,8.42,11.37,0.37,0.26,-0.26,2.95
Romania,9.14,11.9,0.24,0.3,-0.25,2.76
Croatia,9.45,12.18,1.39,0.13,-0.13,2.73


Comparing population_growth and cal_pop_growth shows with some exceptions their values are mostly the same but the population_growth does not show whether the population is increasing or decreasing.

## The highest ratio of population over the area:
It can also be useful to calculate the highest population /area ratio for each country and compare it with the result of *Densely populated countries* section.

In [17]:
%%sql
SELECT 
    name, 
    population, 
    area, 
    CAST(CAST(population AS FLOAT)/CAST(area AS FLOAT) AS Int) population_density
FROM facts
ORDER BY population_density DESC
LIMIT 20;

 * sqlite:///factbook.db
Done.


name,population,area,population_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


Most of the countries in the table above are very small countries like **Macau** and **Monaco** with 28 and 2 square kilometers respectively.

The result is significantly different in comparison with *Densely populated countries* since small countries with the population less than the average can have a large population in a square kilometer. In the *Densely populated countries* section we looked at the countries with the population more than the average which refers to the big countries.

## Conclusion
Analyzing countries according to different factors of population and area achieve the following results:

- **China** with more than 1.3 billion people and the population growth of 0.45 is the most populated country. 
- **South Sudan** with a population of 1.2 million has the highest population growth which is 4.02.
- **India** is the country which will add the most people to the world for the next year (2016). It is estimated to be about 15 million people. **China** and **Nigeria** are at the second and third positions respectively.
- With some exceptions like Japan, most of the countries with a death rate of more than birth rate are European countries. **Bulgaria**, **Serbia** and **Latvia** have the highest net_death_rate.
- **Macau** with about 21 thousand people per 1 square kilometer has the highest population density in the world. **Bangladesh** with the population density of 1138 has the highest density among the countries with the population above the average and the area less than the average.
- **British Indian Ocean Territory** and **Virgin Islands** are the only
places that have more areas of water than land. Some of **Caribean islands**, **Guinea-Bissau**, **Malawi**, and **Netherlands** are the next countries that have the highest ratio of water than land.